In [1]:
from io import StringIO
import requests
import pandas as pd
from bs4 import BeautifulSoup
from  datetime import datetime , timedelta
import bs4
import numpy as np
import re

In [61]:
url = "https://www.moneycontrol.com/india/mutualfunds/toprated/12/00/fundratings"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

In [62]:
elements = soup.find_all(class_="bl_12")
top_mf_url = []
for i in elements:
    top_mf_url.append(i['href'])

top_mf_url = top_mf_url[1:-2]

def clean_mutual_fund_name(url):
    name = url.split('/')[-2]
    cleaned_name = re.sub(r'-', ' ', name).title()
    return cleaned_name

top_mf = []
for i in top_mf_url:
    top_mf.append(clean_mutual_fund_name(i))

In [48]:
ratios = []
yr2 = []
yr5 = []
for i in top_mf_url:
    url = "https://www.moneycontrol.com/india/"+i
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    elements = soup.find_all(class_="percentage")
    rat = []
    for j in elements:
        value = float(str(j.span)[6:-7])
        rat.append(value)
    ratios.append(rat) 

    url = "https://www.moneycontrol.com/india/"+i
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    elements = soup.find_all(class_="scheme_per_amt prcntreturn 5Y")
    if elements != []:
        yr5.append(float(elements[0].get_text().strip()[:-1]))
    else:
        yr5.append(-1)

    url = "https://www.moneycontrol.com/india/"+i
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    elements = soup.find_all(class_="scheme_per_amt prcntreturn 2Y")

    if elements != []:
        yr2.append(float(elements[0].get_text().strip()[:-1]))
    else:
        yr2.append(-1)

In [63]:
to_remove = []
for i in range(len(ratios)):
    if len(ratios[i]) < 5 or yr2[i] == -1 or yr5[i] == -1:
        to_remove.append(i)

for i in to_remove[::-1]:
    del top_mf[i]
    del ratios[i]
    del yr2[i]
    del yr5[i]

In [69]:
df = pd.DataFrame(ratios)
df['cagr2'] = yr2
df['cagr5'] = yr5
df['mf'] = top_mf
df.columns = ['std', 'beta', 'sharpe', 'treynor', 'alpha', 'cagr2', 'cagr5', 'mf']

In [73]:
df.to_csv("mf_saved.csv")